Fake News Calssification

In [1]:
# Making necessary imports
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Reading the data and storing it in the dataframe
df = pd.read_csv('fake_or_real_news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
# Gives the corresponding number of rows and columns
df.shape

(6335, 4)

In [4]:
# Now we are going to get the labels from the dataframe
labels = df.label
labels.head()


0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [5]:
from nltk.corpus import stopwords

In [6]:
from nltk.stem.porter import PorterStemmer

In [7]:
import re

In [8]:
port_stem=PorterStemmer()

In [9]:
port_stem

<PorterStemmer>

In [10]:
port_stem.stem("Hi this is news * % @@@")

'hi this is news * % @@@'

In [11]:
def stemming(content):
    con = re.sub('[^a-zA-Z]',' ', content )
    con = con.lower()
    con = con.split()
    con =[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con =''.join(con)
    return con

In [12]:
stemming("Hi this is news")

'hinews'

In [13]:
#from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize
#stemmer = PorterStemmer()

#df['text'] = df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))


In [14]:
df['text'] = df['text'].apply(stemming)

In [15]:
x = df['text']

In [16]:
y = df['text']

In [17]:
y.shape()

TypeError: 'tuple' object is not callable

In [18]:
# Splitting the dataset
x_train, x_test, y_train, y_test = train_test_split(df['text'], labels, test_size=0.2, random_state=10)

In [19]:
# Initializing the TfidfVectorizer
tfidf_vect = TfidfVectorizer(stop_words='english', max_df=0.8)

# Fit and transform train_Set and test_set
tfidf_train = tfidf_vect.fit_transform(x_train)
tfidf_test = tfidf_vect.transform(x_test)

In [20]:
# Initialize a passive Aggressive Classifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [21]:
# Predict the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)
acc_score = accuracy_score(y_test, y_pred)
print("Accuracy: {} % ".format(round(acc_score*100, 2)))

Accuracy: 51.7 % 


In [22]:
# Build a confusion matrix
confusion_matrix(y_test, y_pred, labels=['FAKE', 'REAL'])

array([[603,   0],
       [612,  52]], dtype=int64)

In [23]:
import pickle

In [24]:
pickle.dump(tfidf_vect, open('vector.pkl','wb'))

In [25]:
pickle.dump(pac, open('model.pkl','wb'))

In [26]:
vector_form = pickle.load(open('vector.pkl','rb'))

In [27]:
load_model = pickle.load(open('model.pkl','rb'))

In [28]:
def fake_news(news):
    news = stemming(news)
    input_data = [news]
    vector_form1 = vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return y_pred

In [31]:
val = fake_news("""U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.

Kerry said he expects to arrive in Paris Thursday evening, as he heads home after a week abroad. He said he will fly to France at the conclusion of a series of meetings scheduled for Thursday in Sofia, Bulgaria. He plans to meet the next day with Foreign Minister Laurent Fabius and President Francois Hollande, then return to Washington.

The visit by Kerry, who has family and childhood ties to the country and speaks fluent French, could address some of the criticism that the United States snubbed France in its darkest hour in many years.

The French press on Monday was filled with questions about why neither President Obama nor Kerry attended Sundayâ€™s march, as about 40 leaders of other nations did. Obama was said to have stayed away because his own security needs can be taxing on a country, and Kerry had prior commitments.

Among roughly 40 leaders who did attend was Israeli Prime Minister Benjamin Netanyahu, no stranger to intense security, who marched beside Hollande through the city streets. The highest ranking U.S. officials attending the march were Jane Hartley, the ambassador to France, and Victoria Nuland, the assistant secretary of state for European affairs. Attorney General Eric H. Holder Jr. was in Paris for meetings with law enforcement officials but did not participate in the march.

Kerry spent Sunday at a business summit hosted by Indiaâ€™s prime minister, Narendra Modi. The United States is eager for India to relax stringent laws that function as barriers to foreign investment and hopes Modiâ€™s government will act to open the huge Indian market for more American businesses.

In a news conference, Kerry brushed aside criticism that the United States had not sent a more senior official to Paris as â€œquibbling a little bit.â€ He noted that many staffers of the American Embassy in Paris attended the march, including the ambassador. He said he had wanted to be present at the march himself but could not because of his prior commitments in India.

â€œBut that is why I am going there on the way home, to make it crystal clear how passionately we feel about the events that have taken place there,â€ he said.

â€œAnd I donâ€™t think the people of France have any doubts about Americaâ€™s understanding of what happened, of our personal sense of loss and our deep commitment to the people of France in this moment of trauma.â€""")

In [32]:
if val ==[0]:
    print('Reliable')
else:
    print('Unreliable')

Unreliable


C:\Users\KIIT\AppData\Local\Temp\ipykernel_17344\637108494.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if val ==[0]:
